TODO
- Fix german and polish word lists
- Check all word lists
- Check uber-alphabet
- Maybe portuguese?
- Another germanic language?
- Print out punctuation and special letters

Overview
====
Parlez-vous français? Laitent illes!

"Laitent illes" is a French expression meaning, of course, nothing, because those aren't real French words. In fact, they are fake French words generated by training an algorithm on a French text. These words look to me, a non-French-speaker, like realistic French words, and hopefully they do to you, too. Of course, if you speak French, they probably look terrible, and you're probably already mad at me. Instead, take a look at some of the other languages that I've modeled!

This project explores using a Markov model to assign likelihoods to words in an alphabet-based language, after being trained on a text in that language. These likelihoods are then used to generate realistic-looking fake words and to detect "foreign-looking" words.

What is a "word?"
----------------
There are a lot of ways to define a "word." For this project, I'll define a word to be a sequence of letters and certain approved punctuation marks. As far as approved punctuation, I have included the apostrophe for most alphabets. This means that a contraction like "don't" will be counted as a single word. This seems better than counting "don't" as two separate words, "don" and "t."

For languages with accented letters, I've chosen to model accented letters as if each one were its own distinct letter. So, my French "alphabet" includes 'E,' 'É,' 'È,' 'Ê,' and 'Ë' as separate "letters." Confusingly, it is possible for a dictionary or text to contain letters that are not in the alphabet! For example, the letters 'J,' 'K,' 'W,' 'X,' and 'Y' are not considered to be part of the Italian alphabet, but they occur frequently in loan words.

Below are the alphabets that I've defined for each language. If I've mauled your alphabet beyond all recognition, please send me an email; I'll be happy to fix it!

In [1]:
# How does anyone survive without these?
%load_ext autoreload
%autoreload 2

In [2]:
from fake_words.fake_words import LANGUAGES

for language in LANGUAGES.values():
    print u"{}: {}".format(language.name, " ".join(list(language.alphabet)))

latin: a b c d e f g h i l m n o p q r s t u v w x z
german: a ä b c d e f g h i j k l m n o ö p q r s ß t u ü v w x y z
spanish: a á b c d e é f g h i í j k l m n ñ o ó p q r s t u ú ü v w x y z
french: a à â ä b c ç d e é è ê ë f g h i î ï j k l m n o ö ô p q r s t u û ü ù v w x y z '
english: a b c d e f g h i j k l m n o p q r s t u v w x y z '
polish: a ą b c ć d e ę f g h i l ł m n ń o ó p r s ś t u x y z ź ż
italian: a à b c d e è é f g h i ì l m n o ò p q r s t u ù v z '


A first pass: letter frequency
================
What constitutes a good fake word? "Mait" seems like a good fake French word to me, whereas "xkzz" does not seem like a good fake French word. How can we differentiate between these words?

First up: letter frequency. 'R,' 'A,' 'I,' and 'T' are all commmon letters in French, whereas 'X,' 'K,' and 'Z' are not. This can be used to create a simple language model wherein words with common letters are judged to be likely, and words with uncommon letters are judged to be unlikely.

For my first model, I'll define the likelihood of a word to be the product of the likelihood of each letter in the word. In this model, there is one parameter for each letter of the alphabet, which will be the frequency of that letter in the language.

In order to learn these parameters, I have included a text for each language. It's possible to get maximum likelihood estimates for each letter frequency by counting the number of times that letter occurs, out of the total number of characters in the training document.

This model makes it very easy for us to do computations, because all letters are independent of one another! Almost... *too* easy.

Here's a short method to print out the most likely words for each language, for each length of word. Since this model models word likelihood as a product of letter likelihoods, shorter words will tend to have higher likelihoods. In any language, the most likely word will be the empty string.

Since an *M*-letter alphabet can form *M*^*N* words of length *N*, exploring the search space of all words is expensive when *N* is large. I have (slightly) optimized this search by pruning the search if the likelihood of the word I'm at is already lower than the likelihood of one of the top words that's already been found. I tried to create a heuristic pruning strategy, but it did not work well.  With my current search, though, I can only generate words of up to four letters in a sane amount of time.

In [3]:
MAX_WORD_LENGTH = 4

In [4]:
from fake_words.fake_words import Language

def print_for_max_gram(min_gram, max_gram):
    languages = [Language(info, min_gram, max_gram) for info in LANGUAGES.values()]
    
    for language in languages:
        for word_length in range(1, MAX_WORD_LENGTH + 1):        
            top_words = language.top_words(word_length, 10)
            top_words_formatted = u", ".join(top_words)
            print u"{}-gram {} length-{}: {}".format(max_gram, language.info.name, word_length, top_words_formatted)
            print
        print

All right, here we go...

In [5]:
print_for_max_gram(0, 1)

1-gram latin length-1: e, i, t, a, u, s, n, o, r, m

1-gram latin length-2: ee, ei, ie, ii, et, te, it, ti, ea, ae

1-gram latin length-3: eee, iee, eei, eie, iie, eii, iei, iii, ete, tee

1-gram latin length-4: eeee, eiee, eeie, eeei, ieee, eiie, eeii, ieie, eiei, iiee


1-gram german length-1: e, n, i, r, s, a, t, h, d, l

1-gram german length-2: ee, ne, en, ei, ie, er, re, es, se, ae

1-gram german length-3: eee, ene, een, nee, iee, eei, eie, eer, ere, ree

1-gram german length-4: eeee, neee, eeen, eene, enee, eeie, eeei, eiee, ieee, reee


1-gram spanish length-1: e, a, o, s, n, r, i, l, d, t

1-gram spanish length-2: ee, ae, ea, aa, eo, oe, ao, oa, se, es

1-gram spanish length-3: eee, aee, eae, eea, eaa, aae, aea, aaa, eeo, oee

1-gram spanish length-4: eeee, aeee, eeae, eeea, eaee, eaae, eaea, eeaa, aaee, aeae


1-gram french length-1: e, a, i, t, s, n, r, u, l, o

1-gram french length-2: ee, ea, ae, ei, ie, et, te, se, es, ne

1-gram french length-3: eee, aee, eae, eea, eei, ei

...and it's terrible. These do not look like words at all. Nevertheless, there are a few useful things to be gleaned from this.

Since a length-1 word is the same as a letter, the length-1 words serve as a double-check on the per-letter parameters. The English letters with highest likelihoods are, in order, "ETAONI." This is close to the generally-accepted list of most frequent English letters, "ETAOIN." The discrepancy may be because I'm using a slightly old text (*A Tale of Two Cities*), or just because the text is too short. Overall, the letter frequency ordering looks good for most languages.

Another interesting thing to note here is that letter order does not matter. For example, after 'eee,' 'eet,' 'tee,' and 'ete' are all tied for second place in English.

There is a very glaring problem with this approach. Although the letter 'E' is very common in English, it is very uncommon to have a word that is composed exclusively of the letter 'E.' To solve this problem, we need a model that models dependencies between letters!

Using a Markov model
===========
I have chosen to use a Markov model to model the interactions between letters. Instead of modeling each letter individually, now each a word is modeled as a sequence of states, where each state corresponds to a letter.

Train using... MLE?

Modeling transitions between letters will create a lot of cases where the parameters aren't covered by the training data. For example, *A Tale of Two Cities* doesn't contain the letter sequences "XZ", "ZX," or "VF," amongst many others. In order to prevent this from zeroing out the likelihoods, I've used [add-one smoothing](https://en.wikipedia.org/wiki/Additive_smoothing), adding one to any bigram that doesnt appear in the training text.

I'm handling the starts and ends of words specially to make the model generate more realistic words. For example, the model should capture that it's extremely uncommon for an English word to end with the letter "Q." To do this, I added Markov chain states for start and end tokens. These behave more or less like letters, except that they must appear at the start and the end of each word, and they may not appear in the middle of the words. So, the word "in" is represented as, ```["start token", "I", "N", "end token"]```.

Just because I'm using the bigram model, doesn't mean that I need to stop using the 1-gram model. For the words below, I've multiplied in their 1-gram likelihoods.

In [6]:
print_for_max_gram(0, 2)

2-gram latin length-1: a, i, c, e, p, s, t, q, m, n

2-gram latin length-2: er, is, in, at, es, en, it, am, ti, qu

2-gram latin length-3: ere, ati, eri, ate, ter, int, iti, tis, ser, ite

2-gram latin length-4: erer, ater, eres, atis, eren, eris, iter, iser, atin, erin


2-gram german length-1: d, s, e, w, i, a, g, u, h, m

2-gram german length-2: en, er, de, ei, se, in, st, es, ie, si

2-gram german length-3: den, der, end, sen, dei, ser, ein, ien, ene, ier

2-gram german length-4: dend, enen, ener, dein, ende, dene, eien, eier, send, sten


2-gram spanish length-1: e, d, l, a, c, p, s, m, y, t

2-gram spanish length-2: en, de, es, er, la, el, do, as, ar, an

2-gram spanish length-3: den, des, der, del, ent, ela, era, ese, eno, ere

2-gram spanish length-4: dent, dela, dera, esen, dese, eses, deno, eren, dere, eres


2-gram french length-1: d, l, c, p, s, e, a, m, q, t

2-gram french length-2: le, de, es, la, en, ai, an, ce, ll, se

2-gram french length-3: les, des, len, den, lai, le

All right, much better! To me, these are starting to look much more realistic. To kick it up a notch, here is a Markov chain with a memory of two states — that is, a model that looks at trigrams in addition to bigrams and 1-grams. Bam?

In [7]:
print_for_max_gram(0, 3)

3-gram latin length-1: m, x, w, v, u, r, g, o, c, l

3-gram latin length-2: qu, in, co, pr, no, re, et, se, es, de

3-gram latin length-3: que, qui, qua, con, quo, pro, est, ina, int, non

3-gram latin length-4: cons, quam, coni, quae, cone, quid, quis, esse, sent, quod


3-gram german length-1: x, h, ä, z, y, w, v, ü, u, t

3-gram german length-2: de, un, er, ge, si, di, ei, se, da, zu

3-gram german length-3: und, der, ein, den, ich, die, gen, sei, ine, sch

3-gram german length-4: eine, unde, sein, icht, sich, dern, iche, eind, scht, ders


3-gram spanish length-1: y, á, x, w, k, ú, r, i, p, ó

3-gram spanish length-2: de, la, co, qu, en, es, el, se, un, lo

3-gram spanish length-3: que, con, ent, des, del, est, com, las, lar, los

3-gram spanish length-4: ente, dest, ques, quer, ento, quel, enta, esta, dent, desp


3-gram french length-1: z, e, u, h, q, p, m, è, k, é

3-gram french length-2: de, qu, le, la, et, ce, pa, il, un, l'

3-gram french length-3: que, les, des, qui, ent, qu

One very strange fake word that I notice here is "scht," in fake-German. This doesn't look like a good German word to me, because it does not have any vowels. Since the model never looks at the word as a whole, it has no way of "counting" the vowels, consonants, or any other category of letter to ensure that they are occuring in appropriate proportions.

Generating fake words, finally!
=================
A functional system to model word probabilities can be used for many different things. To start with, we can find fake words, which are sequences of letters that look realistic, but aren't found in a dictionary.

I'll just repeat the previous exercise, but using a dictionary for each language to eliminate sequences of letters that are already real words.

In [8]:
min_gram = 0
max_gram = 3

trigram_languages = [Language(info, min_gram, max_gram) for info in LANGUAGES.values()]

In [9]:
from math import log

for language in trigram_languages:
    for word_length in range(2, MAX_WORD_LENGTH + 1):        
        top_words = language.top_words(word_length, 100)
        top_nonwords = [w for w in top_words if w not in language.dictionary]
        top_nonwords_formatted = u", ".join(top_nonwords[:12])
        print u"{}-gram {} length-{}: {}".format(max_gram, language.info.name, word_length, top_nonwords_formatted)
    print

3-gram latin length-2: qu, co, th, po, di, pe, cu, ho, pa, ma, pu, ta
3-gram latin length-3: con, ina, int, ine, ess, ing, inc, ati, the, vid, ant, pra
3-gram latin length-4: sent, nons, pere, inte, quir, quiu, rest, cont, atis, ment, atio, estr

3-gram german length-2: de, un, ge, si, di, ei, se, wa, we, ih, wi, ni
3-gram german length-3: sei, ine, sch, ern, ung, sid, dem, ers, war, eit, nic, ber
3-gram german length-4: unde, icht, sich, dern, iche, eind, scht, ders, dend, eing, sche, dens

3-gram spanish length-2: co, qu, es, pa, po, pr, di, ma, sa, an, vi, ap
3-gram spanish length-3: ent, est, com, der, dec, den, enc, una, ela, end, esp, cos
3-gram spanish length-4: dest, ques, quer, ento, quel, esta, dent, desp, dela, desa, cona, cont

3-gram french length-2: qu, pa, l', co, so, vo, po, mo, ch, av, pr, to
3-gram french length-3: ent, qu', ill, deu, ett, com, qua, dev, ava, tou, res, dem
3-gram french length-4: mait, dest, ille, ques, less, deur, quis, dess, rent, qu'i, dant, entr



The less familiar I am with a language, the better the fake words look.

Sampling words progressively
===============
What's the fun of making fake words if they can only be four letters long? Especially with German? As I mentioned above, the challenge is that the search space of *N*-letter words is too big when *N* > 4. With Latin's measly 23-letter alphabet, there are only six million possible five-letter words. French, though, with 50 total letters, allows 312 million five-letter words! So, instead of trying to cover the entire search space of long words, why not sample?

For my first sampler, I sample words progressively from left to right, beginning with the start token and sampling each successive letter conditioned on the letters to its left. It should then be possible to generate *N*-letter words by doing rejection sampling. 

In [10]:
for language in trigram_languages:
    print u"{}: {}\n".format(language.info.name, u", ".join(language.sample_progressive(10)))

latin:  inatissulan...,  essetionera...,  rentumeress...,  uterisimata...,  fuemetereti...,  demoriamese...,  atumestitit...,  inateremuse...,  itatiusasem...,  etististore...

german:  sierendenic...,  eindereinse...,  senennenend...,  seinenderne...,  fanneneinen...,  erenesichne...,  kamitendene...,  ichinereind...,  icherernene...,  senenderten...

spanish:  entalensala...,  desiadosoli...,  nicosasaran...,  allaradaser...,  aseriosasad...,  asalasterio...,  alasarestes...,  parosolosan...,  llareciaser...,  esestantant...

french:  sistertesta...,  etterestest...,  gestiontait...,  deurentessa...,  entelessest...,  pardansiter...,  testeraites...,  seunestreue...,  vaitestente...,  nertentente...

english:  sontlyinged...,  ungedoortor...,  thereareret...,  seelendesse...,  itheanother...,  tonederende...,  ingetherett...,  lowedsonere...,  oftedereand...,  thereateand...

polish:  poszystarys...,  niederaniec...,  siałonarych...,  narobioniem...,  torzamiecha...,  maramieniel...,

This is definitely not going to work. Unfortunately, the sampler tends to end up in stationary distribution rather than reaching the end state, which means that it produces a bunch of words that are way too long. This reveals a weakness of using a Markov model: it doesn't know anything about the overall length of the word, so it doesn't get the hint that maybe it would be a good idea to stop adding letters to its already-100-letter-long word.

Sampling words with MCMC
=============
How can we cover our sample space more effectively? This question always seems to have the same answer: Markov Chain Monte Carlo. As such, I made a Metropolis-Hastings sampler that can make two different kinds of moves: it can replace a single letter, or swap two letters.

In [11]:
from operator import itemgetter

for language in trigram_languages:
    for word_length in xrange(1, 13):
        samples, top = language.sample_mh(word_length, max_to_store=6, n_runs=100, n_samples=1000*word_length)

        samples_joined = u", ".join(word for word, prob in samples[:6])
        print u"{}-letter {} samples: {}".format(word_length, language.info.name, samples_joined)
        
        top_joined = ", ".join(word for prob, word in reversed(sorted(top, key=itemgetter(0))))
        print u"{}-letter {} top: {}\n".format(word_length, language.info.name, top_joined)
    print

1-letter latin samples: l, u, q, b, g, m
1-letter latin top: w, v, x, r, u, o

2-letter latin samples: ul, ho, un, po, di, pu
2-letter latin top: qu, in, co, pr, no, re

3-letter latin samples: pra, ins, inf, viu, imi, neq
3-letter latin top: que, qui, qua, con, quo, pro

4-letter latin samples: prop, diss, pati, iums, hoss, coni
4-letter latin top: cons, quam, coni, quae, cone, quid

5-letter latin samples: estar, perit, ferva, marit, atend, anter
5-letter latin top: quiss, quide, consu, quere, queri, sente

6-letter latin samples: eronis, dicari, tisend, nontib, intudi, atiuss
6-letter latin top: quisse, essent, senter, intere, intent, interi

7-letter latin samples: meratam, perenec, restere, punturb, dissero, meatiam
7-letter latin top: sentere, sentent, senteri, essente, quitati, conisse

8-letter latin samples: nonstior, sesseste, etuanter, quistern, inteditu, lentatus
8-letter latin top: essenter, sentente, quitatis, quitatio, intenter, essentem

9-letter latin samples: venissen

Encouragingly, these sampled words match our exhaustively-searched words well! This indicates that the sampler is probably not completely broken.

These words looks pretty good. Once the words get long, certain (often three-letter) sequences of letters start to repeat. This makes a lot of sense, since the model is only aware of the neighborhood around each letter.

Detecting foreign-looking words
=================
Time for some more fun! For this tangent, I'll go through the dictionary of each language to find words that seem to belong to other languages. Some of these are obviously loan words, but other ones (e.g. for Latin) just look foreign.

In order to get a ranking, I've simply taken the ratio of the likelihood in the destination languages to the likelihood in the source language.

In [12]:
from operator import itemgetter

from fake_words.fake_words import try_to_store

N_TO_STORE = 5

# Loop over every pair of languages
for language_source in trigram_languages:
    for language_dest in trigram_languages:
        # A heap to store our best results
        best_words = []

        if language_dest is language_source:
            continue
    
        for word in language_source.dictionary:
            word_with_tokens = u" {} ".format(word)
            # Subtraction produces a ratio because these are log probabilities
            ratio = language_dest.get_word_prob(word_with_tokens) - language_source.get_word_prob(word_with_tokens)
            try_to_store(best_words, word, ratio, N_TO_STORE)

        best_words_pretty = ", ".join(word for prob, word in reversed(sorted(best_words, key=itemgetter(0))))
        print u"{} words that look {}: {}".format(language_source.info.name, language_dest.info.name, best_words_pretty)

latin words that look german: trochleis, echeneideis, ichneumoneis, andrachnen, echeneides
latin words that look spanish: decharmidabar, decharmidabant, decharmidabas, decharmidabare, decharmidandos
latin words that look french: transfigurans, chondrilles, deurier, transfigurent, brancham
latin words that look english: atheismom, branchad, heirai, chelyin, spathad
latin words that look polish: zelotypiad, chrysopastom, zelotypam, zelotypas, brachypotad
latin words that look italian: leopardale, leopardales, leopardalim, leopardali, leopardaleis
german words that look latin: --------------------------------------------------------------------------, (http://creativecommons.org/licenses/by-nc/3.0/deed.de)., computersimulation, quantitativ, antiquariat
german words that look spanish: parlamentarismus, europaparlament, violoncello, terminologie, casanova
german words that look french: portemonnaie, saisontreffer, europaparlament, violoncello, quantitativ
german words that look english: opp

All right, now I'm having fun!

I always feel slightly offended when I see the words that look English. "Whiskey." "Halloween." "Branchad." "Spathad." "Overbooking!" Is this what English looks like to non-English-speakers? To make myself feel better, I imagine a heavy-set German man with a disproportionately large mustache saying, "braunschweiger," "trochleis," and "weltschmerzes" very seriously.

Hopefully this article has illustrated the strengths and weaknesses of using a Markov model for word structure, but more importantly, I hope that everyone had fun.

Thanks
------
Thanks to my brother Eric for statistics help, and...